# Advent of code 2022

Technically this should probably be modelled with an array or something, but the grid's small enough to use a dictionary and might make things slightly easier:

In [1]:
from collections import defaultdict

import re
import itertools as it

## Part 1

In [2]:
test_input='''Sensor at x=2, y=18: closest beacon is at x=-2, y=15
Sensor at x=9, y=16: closest beacon is at x=10, y=16
Sensor at x=13, y=2: closest beacon is at x=15, y=3
Sensor at x=12, y=14: closest beacon is at x=10, y=16
Sensor at x=10, y=20: closest beacon is at x=10, y=16
Sensor at x=14, y=17: closest beacon is at x=10, y=16
Sensor at x=8, y=7: closest beacon is at x=2, y=10
Sensor at x=2, y=0: closest beacon is at x=2, y=10
Sensor at x=0, y=11: closest beacon is at x=2, y=10
Sensor at x=20, y=14: closest beacon is at x=25, y=17
Sensor at x=17, y=20: closest beacon is at x=21, y=22
Sensor at x=16, y=7: closest beacon is at x=15, y=3
Sensor at x=14, y=3: closest beacon is at x=15, y=3
Sensor at x=20, y=1: closest beacon is at x=15, y=3'''

In [3]:
with open('data/day15.txt') as fIn:
    puzzle_input=fIn.read()

It'll be useful to have a function that returns the coordinates that are a Manhattan distance `x` away:

In [4]:
def coords_from_origin(distance):
    
    if distance==0:
        return[(0, 0)]
    
    out=[]
    for i in range(1, distance):
        out.extend([(i, distance-i), (i, i-distance), (-i, distance-i), (-i, i-distance)])
    out.extend([(0, distance), (0, -distance), (distance, 0), (-distance, 0)])
    
    return out

coords_from_origin(2)

[(1, 1), (1, -1), (-1, 1), (-1, -1), (0, 2), (0, -2), (2, 0), (-2, 0)]

In [5]:
def within_distance(x, y, distance):
    '''return all points within manhattan distance of (x, y)'''
    return [(x+x1, y+y1) for coords in [coords_from_origin(i) for i in range(distance+1)]
                         for (x1, y1) in coords]

within_distance(5, 5, 2)

[(5, 5),
 (5, 6),
 (5, 4),
 (6, 5),
 (4, 5),
 (6, 6),
 (6, 4),
 (4, 6),
 (4, 4),
 (5, 7),
 (5, 3),
 (7, 5),
 (3, 5)]

OK, now let's parse the input:

In [6]:
def day15_a_1(str_in, y_chk):
    
    grid={}

    gd=[[int(i) for i in cc]
        for cc in re.findall('Sensor at x=(\-?\d+), y=(\-?\d+): closest beacon is at x=(\-?\d+), y=(\-?\d+)',
                             str_in)]
    
    for (sx, sy, bx, by) in gd:

        # Get the manhattan distance from the beacon to the sensor
        md=abs(sx-bx)+abs(sy-by)
        
        # And add the covered coords:
        for (x, y) in within_distance(sx, sy, md):
            grid[(x, y)]='#'
    
    # Finally, need to remove the points where there's actually a beacon:
    for (sx, sy, bx, by) in gd:
        grid[(bx, by)]='B'
    
    return len([c for c in grid
                if grid[c]=='#' and c[1]==y_chk])
        

In [7]:
assert day15_a_1(test_input, 10)==26

OK, that's fine, but the grid's going to be far too big to do the puzzle input in the same way.

Thinks...

Actually, we only need to get the coverage for the given row. Probably easiest to work it out as a set of ranges.

So the best way to do that will be with an interval tree. And there's an existing implementation in python (of course...)

In [8]:
import intervaltree

Let's create a function that takes a sensor, a beacon and a y coordinate, and returns the range of points covered on the x axis for that value of y:

In [9]:
def x_coverage(sx, sy, bx, by, y_val):
    
    # Get the manhattan distance from the beacon to the sensor
    md=abs(sx-bx)+abs(sy-by)
    
    return ((sx - (md - abs(sy - y_val))),
            (sx + (md - abs(sy - y_val))))

x_coverage(3, 5, 4, 6, 7)

(3, 3)

OK, need to bear in mind that if the range is back to front (ie. the first number is bigger than the second), then it's not covered.

In [10]:
def day15_a(str_in, y_chk):
    
    gds=[[int(i) for i in cc]
         for cc in re.findall('Sensor at x=(\-?\d+), y=(\-?\d+): closest beacon is at x=(\-?\d+), y=(\-?\d+)',
                              str_in)]
    
    ranges=[x_coverage(*gd, y_chk) for gd in gds]
    ranges=[r for r in ranges if r[0]<=r[1]]
    
    t=intervaltree.IntervalTree()

    # +1 to x_max so that it covers the slice rather than the objects
    # (otherwise it thinks eg. (12-12) is a null range)
    for (x_min, x_max) in ranges:
        t[x_min:x_max+1]=(x_min, x_max)
    
    t.merge_overlaps()
    
    # Remove any beacons present in the row:
    
    for (sx, sy, bx, by) in gds:
        if by==y_chk:
            t.chop(bx, bx+1)

    return sum([(i.end-i.begin) for i in t])

In [11]:
assert day15_a(test_input, 10)==26

In [12]:
day15_a(puzzle_input, 2000000)

4737443

## Part 2

Let's just go through each of the beacons and see whether they appear in the covered zone.

In [13]:
def day15_b(str_in, lim=20):
    
    gds=[[int(i) for i in cc]
         for cc in re.findall('Sensor at x=(\-?\d+), y=(\-?\d+): closest beacon is at x=(\-?\d+), y=(\-?\d+)',
                              str_in)]
    
    for y in range(lim):
        
        if not y%500000:
            print(y)
        
        ranges=[x_coverage(*gd, y) for gd in gds]
        ranges=[r for r in ranges if r[0]<=r[1]]

        t=intervaltree.IntervalTree()

        # +1 to x_max so that it covers the slice rather than the objects
        # (otherwise it thinks eg. (12-12) is a null range)
        for (x_min, x_max) in ranges:
            t[x_min:x_max+1]=(x_min, x_max)
        
        # Top and tail the spans
        t.slice(0)
        t.slice(lim+1)
        
        t.remove_envelop(t.begin(), 0)
        t.remove_envelop(lim+1, t.end())
        
        # And merge them
        t.merge_overlaps(strict=False)
        
        if len(t)==2:
            (x_out, y_out)=(sorted(t)[0].end, y)
            return x_out*4000000+y_out

In [14]:
assert day15_b(test_input, 20)==56000011
    

0


Takes ages to run, but that's OK, I can just run it in my next meeting:

In [15]:
day15_b(puzzle_input, 4000000)

0
500000
1000000
1500000
2000000
2500000


11482462818989

Took about 20 minutes to run. That's OK, it's not like I needed to be at the keyboard.